In [59]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import mean_squared_error as mse 
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer

# Armando un SK Learn Pipeline. 

### Primero las funciones básicas de preprocesamiento. 

In [37]:
def basic_preprocessing(path):
    """
    Una función básica para preprocesar los datos de entrenamiento. 
    Parameters:
    -------
    path: str
          path en tu compu donde está el dataset
    
    Returns:
    --------
    data: pandas dataframe
          Un dataframe listo para el pipeline de sklearn. 
    """
    data = pd.read_csv(path)
    data['MS SubClass'] = data['MS SubClass'].astype('category')
    data = data[data['Sale Condition'] == "Normal"]
    data = data[data['Gr Liv Area'] < 4_000]
    return data 

Es mejor no hacer transformaciones de columnas aquí porque eso lo haremos en el pipeline. 

### Después iniciamos el pipeline. 

No está tan sencillo crearlo pero tampoco es el infierno. Primero necesitamos crear una clase con algunas transformaciones básicas. 

No voy a tomar todas las columnas, es solo un ejemplo. 

In [41]:
pd.set_option('display.max_columns', 40)
data = basic_preprocessing('casas_entrena.csv')
data.head()

,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,...,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,120,RM,NaN,3072,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Blmngtn,Norm,Norm,TwnhsE,1Story,7,5,2004,2004,...,2,388,TA,TA,Y,143,20,0,0,0,0,NaN,NaN,NaN,0,9,2006,WD,Normal,225000
2,120,RL,43.0,3013,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Blmngtn,Norm,Norm,TwnhsE,1Story,7,5,2005,2005,...,2,440,TA,TA,Y,142,20,0,0,0,0,NaN,NaN,NaN,0,4,2006,WD,Normal,213490
4,120,RL,NaN,3196,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Blmngtn,Norm,Norm,TwnhsE,1Story,8,5,2003,2003,...,2,400,TA,TA,Y,143,20,0,0,0,0,NaN,NaN,NaN,0,5,2006,WD,Normal,215000
6,20,RL,43.0,3182,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Blmngtn,Norm,Norm,1Fam,1Story,7,5,2007,2007,...,2,388,TA,TA,Y,100,16,0,0,0,0,NaN,NaN,NaN,0,3,2008,WD,Normal,159895
7,120,RL,43.0,3203,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Blmngtn,Norm,Norm,TwnhsE,1Story,7,5,2006,2006,...,2,437,TA,TA,Y,100,116,0,0,0,0,NaN,NaN,NaN,0,1,2010,WD,Normal,160000


In [50]:
#con esto puedes ver el índice de alguna columna en particular 
data.columns.get_loc("Year Built")

18

In [77]:
class CombinedAttributesAdder(BaseEstimator, TransformerMixin): #dejar esto así
    def __init__(self, house_condition = True): #te permite hacer pruebas 
        self.house_condition = house_condition
    def fit(self, X, y=None):
        return self #no hay que hacer nada más 
    def transform(self, X, y=None):
        #supongamos que queremos calidad de la casa al cuadrado
        qual_squared = X[:, 1] ** 2
        #y otra que sea calidad * metros cuadrados
        qual_m2 = X[:, 1] * X[:,0]
        if self.house_condition: #pudo haber sido el año o alguna otra variable 
            cond = X[:,2]
            return np.c_[X, qual_squared, qual_m2, cond]
        else:
            return np.c[X, qual_squared, qual_m2]

### Ahora se entenderá más claro qué es esto. 

Básicamente, esta clase nos ayuda a definir las nuevas variables que queremos agregar y es algo que usaremos en nuestro pipeline. El siguiente paso es crear un pipeline para las variables numéricas. 

In [78]:
 data.columns[data.isna().any()].tolist()
#columnas con datos vacíos

['Lot Frontage',
 'Alley',
 'Mas Vnr Type',
 'Mas Vnr Area',
 'Bsmt Qual',
 'Bsmt Cond',
 'Bsmt Exposure',
 'BsmtFin Type 1',
 'BsmtFin Type 2',
 'Electrical',
 'Fireplace Qu',
 'Garage Type',
 'Garage Yr Blt',
 'Garage Finish',
 'Garage Qual',
 'Garage Cond',
 'Pool QC',
 'Fence',
 'Misc Feature']

In [79]:
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler()),
])

## ¿Cómo se usa esto? 

Supongamos que importo el dataset y me quedo solo con unas 5 variables. 

In [80]:
data_demo = data[['Lot Area', 'Overall Qual', 'Overall Cond', 'Year Built', 'Garage Area']]

In [81]:
data_demo.head()

,Lot Area,Overall Qual,Overall Cond,Year Built,Garage Area
0,3072,7,5,2004,388
2,3013,7,5,2005,440
4,3196,8,5,2003,400
6,3182,7,5,2007,388
7,3203,7,5,2006,437


In [82]:
data_prepared = numeric_pipeline.fit_transform(data_demo)

# Placeholder: 

* Súper buen progreso, se ve interesante. 
* Encontrar un workaround para los índices y la selección de variables
* Crear un nuevo pipeline para las categóricas y chance uno para el año
* Seguir documentando este ejemplo :) para llegar a algo sólido. 